I had some code to convert DCC experiments to our identifiers. But it was specific to another application. Lets reuse that notebook. to answer that question for some other experiments.

In [1]:
import pandas
import os
import sys
import RDF
from IPython.display import display, HTML

from curation_common import *

In [2]:
%load_ext rdfmagic

The rdfmagic extension is already loaded. To reload it, use:
  %reload_ext rdfmagic


In [3]:
server = ENCODED("www.encodeproject.org")
server.load_netrc()

In [4]:
model = get_model()

In [5]:
experiments = [
  '/experiments/ENCSR000OXO/',
  '/experiments/ENCSR268AXN/',
  '/experiments/ENCSR592MSV/',
]

In [6]:
query = None
if query:
    experiments.extende([ x['@id'] for x in server.get_json(query)['@graph'] ])

# Load Experiments from ENCODED search

In [7]:
for experiment in experiments:
    print('loading {}'.format(experiment))
    details = server.get_jsonld(experiment)
    load_jsonld_into_model(model, details)

loading /experiments/ENCSR000OXO/
loading /experiments/ENCSR268AXN/
loading /experiments/ENCSR592MSV/


# Convert alias to Jumpgate Library URL

This is a construct query whose purpose is to annotate the DCC library with a Wold-lab linked data identifier.

In [8]:
q = RDF.SPARQLQuery("""
PREFIX htsw: <http://jumpgate.caltech.edu/wiki/LibraryOntology#>
PREFIX barbara-wold: <http://jumpgate.caltech.edu/wiki/LibraryOntology#>
PREFIX experiment: <https://www.encodeproject.org/profiles/experiment.json#> 
PREFIX library: <https://www.encodeproject.org/profiles/library.json#> 
PREFIX replicate: <https://www.encodeproject.org/profiles/replicate.json#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

construct { 
  ?library htsw:library ?jumpgate . 
} where {
  select ?experiment?alias
         ( REPLACE(?alias, "barbara-wold:", "http://jumpgate.caltech.edu/library/") as ?short )
         ( IRI(CONCAT(?short, "/"))  as ?jumpgate )
  where {
    ?library library:aliases ?alias .
  }
}
""")

Attach our library urls directly to the encode library objects, and while we have it we might as well load the additional metadata from our db.

In [9]:
results = q.execute(model)
for s in results.as_stream():
    jumpgate = s.object
    load_into_model(model, 'rdfa', jumpgate)
    model.append(s)
    

# Show woldlab library description with ENCODED accessions

In [10]:
%%sparql -m model -o report
select distinct ?experiment ?library ?jumpgate ?name
where {
  ?library library:aliases ?alias .
  ?replicate replicate:library ?library .
  ?experiment experiment:replicates ?replicate .
  ?library htsw:library ?jumpgate . 
  ?jumpgate htsw:name ?name .
}
order by ?jumpgate


/usr/lib/python2.7/dist-packages/RDF.py:2014: RedlandWarning: Variable alias was bound but is unused in the query
  results = Redland.librdf_query_execute(self._query,model._model)


In [11]:
report

experiment,library,jumpgate,name
experiments:ENCSR000OXO/,libraries:ENCLB917PKP/,library:14653/,Illumina index__710_503_Paired_ends__e11.5 forebrain 1_GITC_7/1/2014_13pgs
experiments:ENCSR000OXO/,libraries:ENCLB026BHP/,library:14654/,Illumina index__711_504_Paired_ends__e11.5 forebrain 2_GITC_7/1/2014_13pgs
experiments:ENCSR592MSV/,libraries:ENCLB449LBZ/,library:14655/,Illumina index__712_505_Paired_ends__e11.5 liver 1_GITC_7/1/2014_13pgs
experiments:ENCSR592MSV/,libraries:ENCLB905LVV/,library:14656/,Illumina index__701_506_Paired_ends__e11.5 liver 2_GITC_7/1/2014_13pgs
experiments:ENCSR268AXN/,libraries:ENCLB893OOW/,library:15245/,Illumina index__508_705_Paired_ends__LCMD_711_Mm_C57Bl6_8mo_WT_pyramidal_neurons_50pool_
experiments:ENCSR268AXN/,libraries:ENCLB730HDZ/,library:15246/,Illumina index__517_706_Paired_ends__LCMD_712_Mm_C57Bl6_8mo_WT_pyramidal_neurons_50pool_


Save report

In [12]:
with open('/tmp/experiment.html', 'wt') as outstream:
    outstream.write(os.linesep.join(list(report.generate_html())))